定义获取显存占用的函数，返回总占用和当前行为占用

In [1]:
import torch
import torch.nn.functional as F

last_gpu_memory = 0

def get_memory():
  global last_gpu_memory
  last = last_gpu_memory
  now = torch.cuda.memory_allocated()/1024/1024
  last_gpu_memory = now
  return now,now-last
print(get_memory())


(0.0, 0.0)


用于分析显存占用的模型，包括三个线性层，一个激活层，一个softmax层

In [2]:
class BasicModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.lr1 = torch.nn.Linear(1024,1024)
    self.relu = torch.nn.ReLU()
    self.lr2 = torch.nn.Linear(1024,1024)
    self.sof = torch.nn.Softmax(dim = -1)
    self.lr3 = torch.nn.Linear(1024,2048)

  def forward(self,x):
    x = self.lr1(x)
    x = self.relu(x)
    x = self.lr2(x)
    x = self.sof(x)
    x = self.lr3(x)
    return x

测试推断阶段显存占用

In [3]:
with torch.no_grad():
  model = BasicModel().cuda()
  print('mymodel: ',get_memory())
  data = torch.zeros(10240,1024).to('cuda')
  print('input: ',get_memory())
  out = model(data)
  print("output and intermediate: ",get_memory())
print(torch.cuda.max_memory_allocated()/1024/1024)

mymodel:  (16.015625, 16.015625)
input:  (56.015625, 40.0)
output and intermediate:  (144.140625, 88.125)
185.140625
